In [ ]:
import pandas as pd
import sys
import logging
import os
import datetime as dt
sys.path.append('../src')
from utils.fetch_data import fetch_nwis_data
import utils.duckdb_utils as du
import utils.site_list as sl



In [ ]:
# Configure logging ------------------------------------------------
os.makedirs('logs', exist_ok=True)
log_name = 'logs/' + dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log'
logging.basicConfig(filename=log_name,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
# -------------------------------------------------------------------

In [ ]:
print(sl.nwis_sites)
sites = ", ".join(sl.nwis_sites)

In [ ]:
nwis_rename_map = {
    'site_no': 'site_cd',
    'station_nm': 'site_nm',
    'agency_cd': 'agency_cd',
    'dec_lat_va': 'lat_dd',
    'dec_long_va': 'lon_dd',
    'alt_va': 'elev_ft',
    'site_tp_cd': 'site_type'}

In [ ]:
nwis_metadata = fetch_nwis_data(
    site=sites,
    service_code='site')

selected = nwis_metadata[list(nwis_rename_map)].rename(columns=nwis_rename_map).reset_index(drop=True)
merged = selected.merge(sl.hydrologic_areas, on='site_cd', how='left')
merged['elev_m'] = (merged['elev_ft'] * 0.3048).round()
merged['lat_dd'] = merged['lat_dd'].round(7)
merged['lon_dd'] = merged['lon_dd'].round(7)
merged['agency_nm'] = 'US Geological Survey'
merged['site_type'] = merged['site_type'].str.replace('ST', 'Stream')
merged['site_type'] = merged['site_type'].str.replace('LK', 'Lake')
merged['source'] = 'NWIS API'
merged['site_dsc'] = None
merged['site_id'] = range(1, len(merged) + 1)
merged['create_ts'] = dt.datetime.now()
merged['update_ts'] = dt.datetime.now()

nwis_metadata_cleaned = merged[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']].sort_values(by='site_cd')


In [ ]:
du.run_sql_file('../db/schema.sql')

In [ ]:
query = """
INSERT INTO site
SELECT * FROM staging_table
"""
print(query)

In [ ]:
with du.connect_duckdb() as con:
    columns = con.execute("PRAGMA table_info('site')").fetchall()
    print(columns)

In [ ]:
with du.connect_duckdb() as con:
        try:
            con.register('staging_table', nwis_metadata_cleaned)
            con.execute(query)
            logging.info(f"✅ Successfully executed SQL query: {query}")
        except Exception as e:
            logging.error(f"❌ Error executing SQL file {query}: {e}")
            raise


In [ ]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT COUNT(*) FROM site").fetchone()
    print(f"Number of rows in 'site' table: {result[0]}")
    logging.info(f"Number of rows in 'site' table: {result[0]}")


In [ ]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT * FROM site").df()